# Load the Dependencies 

In [1]:
from dotenv import load_dotenv

load_dotenv()

# Model Imports
from langchain_mistralai import ChatMistralAI
from langchain_mcp_adapters.client import MultiServerMCPClient

# Agent Imports
from langchain.agents import create_agent, AgentState
from langchain.messages import HumanMessage, ToolMessage
from langchain.tools import ToolRuntime, tool

from typing import Dict, Any

from tavily import TavilyClient
from langchain_community.utilities import SQLDatabase

from langgraph.types import Command
from langgraph.checkpoint.memory import InMemorySaver
from langchain.tools import ToolRuntime

## Configure the MCP server Launch settings

In [2]:
import sys
import asyncio

# Fix for Windows issues in Jupyter notebooks
if sys.platform == "win32":
    # 1. Use ProactorEventLoop for subprocess support
    if not isinstance(asyncio.get_event_loop_policy(), asyncio.WindowsProactorEventLoopPolicy):
        asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())
    
    # 2. Redirect stderr to avoid fileno() error when launching MCP servers
    if "ipykernel" in sys.modules:
        sys.stderr = sys.__stderr__


In [3]:
# this for the transportation options tool
traviling_client = MultiServerMCPClient(
    {
        "travel_server": {
                "transport": "streamable_http",
                "url": "https://mcp.kiwi.com"
            }
    }
)

the below of the second cell worked because “relative” does not mean “relative to the file you’re editing”.
It means relative to the current working directory (CWD) of the running Python process.

Once that distinction is clear, everything you observed becomes fully consistent.

# Create Agent State

In [4]:
class WeddingState(AgentState):
    origin: str
    destination: str
    guest_count: str
    genre: str

# Core Function Definitions

In [5]:
# First Cell

import os

print("CWD:", os.getcwd())
# here the path is not correct, becuase we are in notebooks/module-2/Wedding_Project_Planners
# so the code tries to find the DB in this directory
# while DB in resources directory, so we cant access it directly, we need to go back two levels
# so we need to change the path to ../resources/Chinook.db as shown in the second cell
print("DB exists:", os.path.exists("notebooks/module-2/Wedding_Project_Planners"))


CWD: c:\Users\abdel\OneDrive\Desktop\lca-lc-foundations\notebooks\module-2\Wedding_Project_Planners
DB exists: False


In [6]:
# Second Cell
import os

print("CWD:", os.getcwd())
print("DB exists:", os.path.exists("../resources/Chinook.db"))


CWD: c:\Users\abdel\OneDrive\Desktop\lca-lc-foundations\notebooks\module-2\Wedding_Project_Planners
DB exists: True


# Create subagents with its tools

In [7]:
# Tool1: Transportation Options Tool

travil_tool = await traviling_client.get_tools()

# define the Tavily client
tavily_client = TavilyClient()

# Tool2: Web Search for Wedding Venues
@tool("search_web_venue", description="Search for wedding venues on the web.")
def tool1(query: str) -> Dict[str, Any]:
    results = tavily_client.search(query=query, num_results=3)
    return {"results": results} # in my case, I will return the whole results dictionary to Gemini


db = SQLDatabase.from_uri("sqlite:///../resources/Chinook.db")
# Tool3: Database Query Tool
@tool("search_playlist", description="Search for playlists in the music database.")
def tool2(query: str) -> str: 
    
    try:
        return db.run(query)
    except Exception as e:
        return f"Error querying database: {e}"


In [8]:
print([t.name for t in travil_tool])

['search-flight', 'feedback-to-devs']


In [9]:
travel_model = ChatMistralAI(
                        model ="mistral-small-latest",
                        temperature=0)
search_model = ChatMistralAI(
                        model ="mistral-small-latest",
                        temperature=0)
playlist_model = ChatMistralAI(
                        model ="mistral-small-latest",
                        temperature=0)  

cordinator_model = ChatMistralAI(
                        model ="mistral-large-latest",
                        temperature=0)  


travel_agent = create_agent(
    model = travel_model,
    tools = travil_tool,
    system_prompt= """ 
    You are a travel agent to Kiwi.com tools. Search for flights to the desired destination wedding location.
    You are not allowed to ask any more follow up questions, you must find the best flight options based on the following criteria:
    - Price (lowest, economy class)
    - Duration (shortest)
    - Date (time of year which you believe is best for a wedding at this location)
    To make things easy, only look for one ticket, one way.
    You may need to make multiple searches to iteratively find the best options.
    You will be given no extra information, only the origin and destination. It is your job to think critically about the best options.
    Once you have found the best options, let the user know your shortlist of options."""
)   


search_agent =create_agent(
    search_model,
    tools = [tool1],
    system_prompt= """ 
    You are a venue specialist. Search for venues in the desired location, and with the desired capacity.
    You are not allowed to ask any more follow up questions, you must find the best venue options based on the following criteria:
    - Price (lowest)
    - Capacity (exact match)
    - Reviews (highest)
    You may need to make multiple searches to iteratively find the best options.
    """ )  

dj_agent =create_agent(
    search_model,
    tools = [tool2],
    system_prompt= """ 
    You are a playlist specialist. Query the sql database and curate the perfect playlist for a wedding given a genre.
    Once you have your playlist, calculate the total duration and cost of the playlist, each song has an associated price.
    If you run into errors when querying the database, try to fix them by making changes to the query.
    Do not come back empty handed, keep trying to query the db until you find a list of songs.
    You may need to make multiple queries to iteratively find the best options.
    """ )


# Main Cordinator

In [10]:
# this only the function signature was changed into async becuase the user of the server MCP.
@tool # no name provided, function name will be used, so the detials in the doc string
async def search_flights(runtime: ToolRuntime) -> str: 
    """ Call the transportation agent to get travel options for desired destination wedding location. """
    origin = runtime.state["origin"]
    destination = runtime.state["destination"]
    response = await travel_agent.ainvoke(
        {"messages": [HumanMessage(content=f"Find me the best flight options from {origin} to {destination} for a wedding.")]}
    )
    return response['messages'][-1].content

@tool # no name provided, function name will be used, so the detials in the doc string
def search_venues(runtime: ToolRuntime) -> str:
    """ Venue agent chooses the best venue for the given location and capacity. """
    destination = runtime.state["destination"]
    capacity = runtime.state["guest_count"]
    response = search_agent.invoke(
        {"messages": [HumanMessage(content=f"Find me the best venue options in {destination} for {capacity} guests for a wedding.")]}
    )
    return response['messages'][-1].content

@tool # no name provided, function name will be used, so the detials in the doc string
def suggest_playlist(runtime: ToolRuntime) -> str:
    """ Playlist agent curates the perfect playlist for the given genre. """
    genre = runtime.state["genre"]
    query = f"Find {genre} tracks for wedding playlist."
    response = dj_agent.invoke(
        {"message": [HumanMessage(content=query)]}
    )
    return response['messages'][-1].content

@tool
def update_state(origin: str, destination: str, guest_count: str, genre: str, runtime: ToolRuntime) -> str:
    """ Update the state when you know all of the values: origin, destination, guest_count, genre. """
    return Command(
        update = 
        {
            "origin": origin,
            "destination": destination,
            "guest_count": guest_count,
            "genre": genre,
            "messages": [ToolMessage("Successfully updated state", tool_call_id=runtime.tool_call_id)]}
    )

In [11]:
cordinator_agent = create_agent(
    cordinator_model,
    tools=[search_flights, search_venues, suggest_playlist, update_state],
    state_schema=WeddingState,
    system_prompt= """ 
    You are a wedding coordinator. Delegate tasks to your specialists for flights, venues and playlists.
    First find all the information you need to update the state. Once that is done you can delegate the tasks.
    Once you have received their answers, coordinate the perfect wedding for me. """
)

# Testing

In [12]:
from langchain.messages import HumanMessage

response = await cordinator_agent.ainvoke(
    {
        "messages": [HumanMessage(content="I'm from London and I'd like a wedding in Paris for 100 guests, jazz-genre")],
    }
)

McpError: MCP error -32602: MCP error -32602: Invalid arguments for tool search-flight: [
  {
    "code": "custom",
    "message": "Dates must be in the future. Current date is 08/01/2026",
    "path": [
      "departureDate"
    ]
  }
]

In [ ]:
print(response["messages"][-1].content  )

Here’s a curated **Jazz wedding playlist** that will set the perfect mood for your Parisian wedding, from the ceremony to the reception:

---

### **🎷 Jazz Wedding Playlist for 100 Guests**

#### **💍 Ceremony**
1. **"La Vie en Rose"** – Louis Armstrong (Classic for the processional)
2. **"What a Wonderful World"** – Louis Armstrong (Uplifting and timeless)
3. **"Fly Me to the Moon"** – Frank Sinatra (Romantic and elegant)
4. **"Cheek to Cheek"** – Ella Fitzgerald & Louis Armstrong (Perfect for the first dance)
5. **"At Last"** – Etta James (A must for weddings)

#### **🍾 Cocktail Hour**
6. **"The Nearness of You"** – Norah Jones (Smooth and intimate)
7. **"Misty"** – Erroll Garner (Jazzy and sophisticated)
8. **"Blue in Green"** – Miles Davis (Instrumental, atmospheric)
9. **"L-O-V-E"** – Nat King Cole (Playful and romantic)
10. **"Dream a Little Dream of Me"** – Diana Krall (Soft and dreamy)

#### **🍽️ Dinner & Reception**
11. **"It Had to Be You"** – Harry Connick Jr. (Upbeat and fun

In [ ]:
response

{'messages': [HumanMessage(content="I'm from London and I'd like a wedding in Paris for 100 guests, jazz-genre", additional_kwargs={}, response_metadata={}, id='4f41accc-4c31-4f2e-b7e0-f73ab01dd216'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'SNaPl6kiO', 'function': {'name': 'update_state', 'arguments': '{"origin": "London", "destination": "Paris", "guest_count": "100", "genre": "jazz"}'}, 'index': 0}]}, response_metadata={'token_usage': {'prompt_tokens': 346, 'total_tokens': 379, 'completion_tokens': 33}, 'model_name': 'mistral-large-latest', 'model': 'mistral-large-latest', 'finish_reason': 'tool_calls', 'model_provider': 'mistralai'}, id='lc_run--019b9d4d-968e-7020-9979-bf657dd957a4-0', tool_calls=[{'name': 'update_state', 'args': {'origin': 'London', 'destination': 'Paris', 'guest_count': '100', 'genre': 'jazz'}, 'id': 'SNaPl6kiO', 'type': 'tool_call'}], usage_metadata={'input_tokens': 346, 'output_tokens': 33, 'total_tokens': 379}),
  ToolMessage(content='S